In [5]:
def standardize(arr):
    arr = np.array(arr)
    arrays = []
    mean = np.sum(arr)/len(arr)
    sd = np.sqrt(np.sum((arr - mean)**2)/len(arr))
    rescaled_and_zeroed = (arr - mean)/sd
    return rescaled_and_zeroed

def normalize(arr):
    arr = np.array(arr)
    normalized = (arr - np.min(arr))/(np.max(arr) - np.min(arr))
    return normalized

def preprocess_cities(cities, standardize=False):
    xs, ys = [], []
    for city in cities:
        xs.append(city[0])
        ys.append(city[1])
    if standardize:
        xs = standardize(xs)
        ys = standardize(ys)
    else:
        xs = normalize(xs)
        ys = normalize(ys)
    new_cities = []
    for i in range(len(cities)):
        new_cities.append((xs[i], ys[i]))
    return np.array(new_cities)

import numpy as np
import random
import operator
from IPython.core.debugger import set_trace
import math
import networkx as nx
from random import randint
from IPython.core.debugger import set_trace
from alphazero import self_play, get_optimal_play
import database as db
from model import create_net
from concurrent import futures as mp
from tkinter import Tk, Canvas
import cProfile

screenSize = 700

def create_tsp_graph(num_nodes, seed=None):
    G = nx.Graph()
    
    if seed is not None:
        np.random.seed(seed)
    
    for i in range(num_nodes):
        G.add_node(i, pos=(np.random.uniform(), np.random.uniform()))
        
    if seed is not None:
        np.random.seed()
        
    return G

def draw_graph(G):
    %matplotlib inline
    import matplotlib.pyplot as plt
    nx.draw(G, nx.get_node_attributes(G, "pos"), node_color="black", linewidths=1, node_size=50)
    plt.show()

def convert_board_to_path(board, cities):
    path = []
    for idx in board:
        path.append(cities[idx-1])
    return path

# def dist_squared(c1, c2):
#     x1, y1 = c1[0], c1[1]
#     x2, y2 = c2[0], c2[1]
#     xdiff = x2 - x1
#     ydiff = y2 - y1
#     res = int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)
#     return res

def distL2(c1, c2):
    x1 = c1[0]
    y1 = c1[1]
    
    x2 = c2[0]
    y2 = c2[1]
    
    xdiff = x2 - x1
    ydiff = y2 - y1
    return math.sqrt(xdiff**2 + ydiff**2)

def calc_length(cities, path):
    length = 0
    for i in range(len(path)-1):
        length += distL2(path[i], path[i+1])

    return length

# def board_converter(board):
#     converted_board = []
#     for idx in board:
#         if idx == 0:
#             converted_board.append([0, 0])
#         else:
#             converted_board.append(cities[idx - 1])
#     converted_board = np.array(converted_board)
#     converted_board = np.expand_dims(converted_board, axis=0)
#     return converted_board



from random import randint
from IPython.core.debugger import set_trace
from alphazero import self_play, get_optimal_play
import database as db
from model import create_net
from concurrent import futures as mp

def main():
    # G = create_tsp_graph(100, seed=1)
    # cities = nx.get_node_attributes(G, "pos")

    db.create_tables()
    # loading data
    f = open("datasets/tsp0100.txt", 'r').read().splitlines()
    numCities = f.pop(0)
    cities = np.array([tuple(map(float, coord.split())) for coord in f])
    cities = preprocess_cities(cities)

    def terminal_function(state, node):
        #     if you have a condition to stop early such as maximum number of steps, or a resignation threshold,
        # specify it here. for TSP it might be if the average distance for the given step is above average you
        # resign
        def early_stop_func():
            pass
#             add earla stop function here
        if node["depth"] == len(cities) - 1:
            path = convert_board_to_path(state.board, cities)

            score = calc_length(cities, path)
#             print(path)
#             print(score)

            prev_score = 0
            results = db.select("""SELECT score FROM data""")
            if len(results) > 0:
                for res in results:
                    prev_score += res[0]
                prev_score /= len(results)

            db.insert("""INSERT INTO data VALUES (?)""", (score,))
            if prev_score == 0:
                return 0
            elif score < prev_score:
                print("prev_score: {}, new_score: {}".format(prev_score, score))
                return 1
            else:
                print("prev_score: {}, new_score: {}".format(prev_score, score))
                return -1
        else:
            return None

    def board_converter(board):
        converted_board = []
        for idx in board:
            if idx == 0:
                converted_board.append([0, 0])
            else:
                converted_board.append(cities[idx - 1])
        return converted_board

#     net = load_model("best_model.h5")

    net = create_net(time_steps=len(cities), input_length=2, output_length=len(cities),
                     num_layers=3)
    
#     cProfile.runctx("""self_play(net, terminal_function, board_converter=board_converter, size=len(cities),
#                     move_list=[i + 1 for i in range(len(cities))],
#                     duplicate_moves=False)""", locals=locals(), globals=globals())
    
    net = self_play(net, terminal_function, board_converter=board_converter, size=len(cities),
                    move_list=[i + 1 for i in range(len(cities))],
                    duplicate_moves=False)

    start = time()
    board = get_optimal_play(net)
    path = convert_board_to_path(board, cities)
    print(path)
    length = eval_path(path)
    tottime = time() - start
    print("Found path of length %s in %s seconds" %
          (round(length, 2), round(tottime, 2)))

    # displaying path
    drawPath(path, cities, length)

    # calculating path
#     path, length = algorithm(cities)

################################ DRAWING METHODS #################################


def randColor():
    return "#%06x" % randint(0, 0xFFFFFF)


def drawPath(path, cities, length):
    cities = cities[path]

    msg = "Length*: {:.2E}".format(length)
    canvas.create_text(screenSize / 2, screenSize + 50,
                       text=msg, fill='black',  font=('Helvetica', 20, 'bold'))

    addToCanvas(cities)
    canvas.update()
    root.mainloop()


def addToCanvas(cities):
    min_x = np.min(cities[:, 0])
    min_y = np.min(cities[:, 1])

    max_x = np.max(cities[:, 0])
    max_y = np.max(cities[:, 1])

    for i in range(len(cities)):
        c = cities[i - 1]
        c_next = cities[i]

        scaled_x = (c[0] - min_x) / (max_x - min_x) * screenSize + 20
        scaled_y = (c[1] - min_y) / (max_y - min_y) * screenSize + 20

        scaled_x_next = (c_next[0] - min_x) / (max_x - min_x) * screenSize + 20
        scaled_y_next = (c_next[1] - min_y) / (max_y - min_y) * screenSize + 20

        canvas.create_oval(scaled_x - 4, scaled_y - 4, scaled_x + 4,
                           scaled_y + 4, fill=randColor(), outline='black')
        canvas.create_oval(scaled_x_next - 4, scaled_y_next - 4, scaled_x_next +
                           4, scaled_y_next + 4, fill=randColor(), outline='black')

        canvas.create_line(scaled_x, scaled_y, scaled_x_next,
                           scaled_y_next, fill='black')


root = Tk()
canvas = Canvas(root, width=screenSize + 40,
                height=screenSize + 100, bg='white')
canvas.pack()

In [9]:
from keras import backend as K

In [10]:
net.get_weights()

[array([[ 0.11165671,  0.01663418,  0.03219865, -0.12459742,  0.07373278,
         -0.19997156, -0.16883828, -0.2056098 ,  0.18176879, -0.20303132,
         -0.05506703,  0.01253991,  0.15744688,  0.16229691,  0.18872674,
         -0.09077866,  0.17657791,  0.1684808 , -0.05534434,  0.10979371,
         -0.04123785,  0.10945775,  0.10868044, -0.17354959, -0.029164  ,
         -0.01886453,  0.00496362,  0.07749631, -0.13289642,  0.02162239,
         -0.19071861, -0.19561207,  0.1504329 ,  0.09677945,  0.03664036,
         -0.12489025,  0.03691839, -0.08361429,  0.18258645,  0.21346299,
         -0.15040699, -0.00685239, -0.16624545,  0.19632865,  0.11673434,
         -0.16054633, -0.01171619,  0.09234564,  0.13712038, -0.2133988 ,
         -0.10066294,  0.11733533, -0.21169916, -0.18127748, -0.15657112,
          0.2061974 ,  0.09890281,  0.19723515,  0.20375235,  0.01990353,
          0.19707988, -0.16528204, -0.00797744, -0.11481266, -0.11737927,
         -0.0457682 ,  0.19906317,  0.

In [6]:
main()

  0%|          | 0/1 [00:00<?, ?it/s]

On move 1 of 100
On move 2 of 100
On move 3 of 100
On move 4 of 100
On move 5 of 100
On move 6 of 100
On move 7 of 100
On move 8 of 100
On move 9 of 100
On move 10 of 100
On move 11 of 100
On move 12 of 100
On move 13 of 100
On move 14 of 100
On move 15 of 100
On move 16 of 100
On move 17 of 100
On move 18 of 100
On move 19 of 100
On move 20 of 100
On move 21 of 100
On move 22 of 100
On move 23 of 100
On move 24 of 100
On move 25 of 100
On move 26 of 100
On move 27 of 100
On move 28 of 100
On move 29 of 100
On move 30 of 100
On move 31 of 100
On move 32 of 100
On move 33 of 100
On move 34 of 100
On move 35 of 100
On move 36 of 100
On move 37 of 100
On move 38 of 100
On move 39 of 100
On move 40 of 100
On move 41 of 100
On move 42 of 100
On move 43 of 100
On move 44 of 100
On move 45 of 100
On move 46 of 100
On move 47 of 100
On move 48 of 100
On move 49 of 100
On move 50 of 100
On move 51 of 100
On move 52 of 100
On move 53 of 100
On move 54 of 100
On move 55 of 100
On move 56 of 100
O


100%|██████████| 1/1 [00:19<00:00, 19.06s/it]


On move 99 of 100


RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [1]:
# from collections import namedtuple
# from tqdm import tqdm

# def create_random_samples(num_samples, cities):
#     samples = []
    
#     for i in tqdm(range(num_samples)):
#         sample = random.sample(city_indices, len(city_indices))
#         board = board_converter(sample, cities)
#         length = 0
#         city_path = board[0]
#         for i in range(len(city_path)):
#             length += distL2(city_path[i-1], city_path[i])
#         samples.append({"board": board, "path": sample, "length": length})
#     return samples
 
# def create_mcts_from_samples(samples):
#     while True:
#         sample = samples[-1]
#         other = samples[0]
#         if sample === other:
#             break
#         for i, idx in enumerate(sample["path"]):
#             if other["path"][i] == idx:
#                 visits += 1
#     for i, sample in enumerate(tqdm(samples)):
#         for j, other in samples:
#             if i != j and sample["path"] == other["path"]:
                
#     results = []
#     res = {}
#     for i in tqdm(range(len(samples[0]["path"]))):
#         for j in range(len(samples[0]["path"])):
#             for sample in samples:
#                 if sample["path"][i] == (j+1):
#                     try:
#                         res[(i, j+1)] += 1
#                     except KeyError as e:
#                         res[(i, j+1)] = 1
                        
#     for tup in res
    
#     set_trace()
#     test = "hi"                

# samples = create_random_samples(10000, cities)
# create_mcts_from_samples(samples)